In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms as transforms
from torchvision.datasets import CIFAR10
import torch.nn as nn
from pytorch_lightning.callbacks import Callback
import torch.optim as optim
import sys
import pytorch_lightning as pl
import os


In [ ]:
#adjustable hparams
batch_size = 64
lr = 1e-3
max_epochs = 12
latent_dim = 128
#change this to 'cuda'
device = 'cpu'


dataset_path = '/Users/gpnuser/Documents/Horos Data/Jas_Pycharm/VAE'
torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=False)
np.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, suppress=True, nanstr=None, infstr=None, formatter=None, sign=None, floatmode=None, legacy=None)

In [ ]:

stride = 2
padding = 1
kernel_size = 3
#dont touch this stuff
# Transformations applied on each image => only make them a tensor
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
transform = transforms.ToTensor()
# Loading the training dataset. We need to split it into a training and validation part
train_dataset = CIFAR10(root=dataset_path, train=True, transform=transform, download=True)
pl.seed_everything(42)
train_set, val_set = torch.utils.data.random_split(train_dataset, [45000, 5000])

# Loading the test set
test_set = CIFAR10(root=dataset_path, train=False, transform=transform, download=True)

# We define a set of data loaders that we can use for various purposes later.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True, drop_last=True, pin_memory=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=256, shuffle=False, drop_last=False, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False, drop_last=False, num_workers=4)


def get_train_images(num):
    return torch.stack([train_dataset[i][0] for i in range(num)], dim=0)


In [ ]:
# layer architecture
class Autoencoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=stride, padding=padding),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=stride, padding=padding),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=stride, padding=padding),           # [batch, 48, 4, 4]
            nn.ReLU(),
# 			nn.Conv2d(48, 96, 4, stride=stride, padding=padding),           # [batch, 96, 2, 2]
#             nn.ReLU(),
        )
        self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(96, 48, 4, stride=stride, padding=padding),  # [batch, 48, 4, 4]
#             nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=stride, padding=padding),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=stride, padding=padding),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=stride, padding=padding),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )
        self.loss_array = np.zeros(max_epochs,)
        self.in_tensor_array = np.zeros(max_epochs,)
        self.out_tensor_array = np.zeros(max_epochs,)


    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
  #      plt.imshow(reconstructed[0].permute(1,2,0))
        return latent, reconstructed

    def get_loss(self, batch):
        x, _ = batch  # We do not need the labels
        latent, reconstructed = self.forward(x)
        loss = nn.functional.mse_loss(x, reconstructed)
        loss = torch.mean(loss)
        print('loss is:')
        print(loss)

        return loss

    def training_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.loss_array[self.current_epoch] = loss
        self.log()
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        #implement log loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def represent(self):
        print('encoder is:')
        print(self.encoder)
        print('decoder is:')
        print(self.decoder)
net = Autoencoder()


In [ ]:
if __name__ == "__main__":
    net.represent()
    trainer = pl.Trainer(max_epochs=max_epochs)

    trainer.fit(net, train_loader, val_loader)

In [ ]:
print(net.loss_array)
print(net.loss_array)